##### Any comments or advice would be appreciated :)

## Using Effnet with YOLO (Keras)

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as tfl
from tensorflow.keras import regularizers

from sklearn.model_selection import train_test_split, KFold
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans

import os
from multiprocessing import cpu_count
from tqdm.notebook import tqdm
from sklearn.model_selection import StratifiedKFold
from scipy.stats import pearsonr
from PIL import Image

import glob
import sys
import cv2
import imageio
import joblib
import math
import warnings
import torch
import imagehash
from sklearn.cluster import KMeans
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76

import warnings
warnings.filterwarnings("ignore")

tqdm.pandas()
np.random.seed(0)
tf.random.set_seed(0)

In [ ]:
# the original file
# train = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')
# train['path'] = '../input/petfinder-pawpularity-score/train/' + train['Id'] + '.jpg'
# train.head(3)

In [ ]:
test = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/test.csv')
test['path'] = '../input/petfinder-pawpularity-score/test/' + test['Id'] + '.jpg'

In [ ]:
# feature pre-extracted file - to save time. (R,G,B columns are not used in this notebook)
train = pd.read_csv('../input/train-final/train_final.csv')
train = train.drop('Unnamed: 0', axis=1)
train['path'] = '../input/petfinder-pawpularity-score/train/' + train['Id'] + '.jpg'
train

### YOLOv5 
- get more meta data from the images

In [ ]:
# importing yolov5 without internet 
!mkdir /root/.config/Ultralytics/
!cp ../input/yolo-arial/Arial.ttf /root/.config/Ultralytics/Arial.ttf
yolov5x6_model = torch.hub.load('../input/d/nilavanakilan/yolov5/', 'custom', source='local', force_reload=True, path='../input/ultralyticsyolov5aweights/yolov5x6.pt')

In [ ]:
# Get Image Info
def get_image_info(file_path, plot=False):
    # Read Image
    image = imageio.imread(file_path)
    h, w, c = image.shape
    
    if plot: # Debug Plots
        fig, ax = plt.subplots(1, 2, figsize=(8,8))
        ax[0].set_title('Pets detected in Image', size=16)
        ax[0].imshow(image)
        
    # Get YOLOV5 results using Test Time Augmentation for better result
    results = yolov5x6_model(image, augment=True)
    
    # Mask for pixels containing pets, initially all set to zero
    pet_pixels = np.zeros(shape=[h, w], dtype=np.uint8)
    
    # Dictionary to Save Image Info
    h, w, _ = image.shape
    image_info = { 
        'n_pets': 0, # Number of pets in the image
        'labels': [], # Label assigned to found objects
        'thresholds': [], # confidence score
        'coords': [], # coordinates of bounding boxes
        'x_min': 0, # minimum x coordinate of pet bounding box
        'x_max': w - 1, # maximum x coordinate of pet bounding box
        'y_min': 0, # minimum y coordinate of pet bounding box
        'y_max': h - 1, # maximum x coordinate of pet bounding box
    }
    
    # Save found pets to draw bounding boxes
    pets_found = []
    
    # Save info for each pet
    for x1, y1, x2, y2, treshold, label in results.xyxy[0].cpu().detach().numpy():
        label = results.names[int(label)]
        if label in ['cat', 'dog']:
            image_info['n_pets'] += 1
            image_info['labels'].append(label)
            image_info['thresholds'].append(treshold)
            image_info['coords'].append(tuple([x1, y1, x2, y2]))
            image_info['x_min'] = max(x1, image_info['x_min'])
            image_info['x_max'] = min(x2, image_info['x_max'])
            image_info['y_min'] = max(y1, image_info['y_min'])
            image_info['y_max'] = min(y2, image_info['y_max'])
            
            # Set pixels containing pets to 1
            pet_pixels[int(y1):int(y2), int(x1):int(x2)] = 1
            
            # Add found pet
            pets_found.append([x1, x2, y1, y2, label])

    if plot:
        for x1, x2, y1, y2, label in pets_found:
            c = 'red' if label == 'dog' else 'blue'
            rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, linewidth=2, edgecolor=c, facecolor='none')
            # Add the patch to the Axes
            ax[0].add_patch(rect)
            ax[0].text(max(25, (x2+x1)/2), max(25, y1-h*0.02), label, c=c, ha='center', size=14)
                
    # Add Pet Ratio in Image
    image_info['pet_ratio'] = pet_pixels.sum() / (h*w)

    if plot:
        # Show pet pixels
        ax[1].set_title('Pixels Containing Pets', size=16)
        ax[1].imshow(pet_pixels)
        plt.show()
        
    return image_info

In [ ]:
# # Image Info train
# IMAGES_INFO = {
#     'n_pets': [],
#     'label': [],
#     'coords': [],
#     'x_min': [],
#     'x_max': [],
#     'y_min': [],
#     'y_max': [],
#     'pet_ratio': [],
# }


# for idx, file_path in enumerate(tqdm(train['path'])):
#     image_info = get_image_info(file_path, plot=False)
    
#     IMAGES_INFO['n_pets'].append(image_info['n_pets'])
#     IMAGES_INFO['coords'].append(image_info['coords'])
#     IMAGES_INFO['x_min'].append(image_info['x_min'])
#     IMAGES_INFO['x_max'].append(image_info['x_max'])
#     IMAGES_INFO['y_min'].append(image_info['y_min'])
#     IMAGES_INFO['y_max'].append(image_info['y_max'])
#     IMAGES_INFO['pet_ratio'].append(image_info['pet_ratio'])
    
#     # Not Every Image can be Correctly Classified
#     labels = image_info['labels']
#     if len(set(labels)) == 1: # unanimous label
#         IMAGES_INFO['label'].append(labels[0])
#     elif len(set(labels)) > 1: # Get label with highest confidence
#         IMAGES_INFO['label'].append(labels[0])
#     else: # unknown label, yolo could not find pet
#         IMAGES_INFO['label'].append('unknown')

In [ ]:
# # Add Image Info to Train
# for k, v in IMAGES_INFO.items():
#     train[k] = v

In [ ]:
# Image Info test
IMAGES_INFO = {
    'n_pets': [],
    'label': [],
    'coords': [],
    'x_min': [],
    'x_max': [],
    'y_min': [],
    'y_max': [],
    'pet_ratio': [],
}


for idx, file_path in enumerate(tqdm(test['path'])):
    image_info = get_image_info(file_path, plot=False)
    
    IMAGES_INFO['n_pets'].append(image_info['n_pets'])
    IMAGES_INFO['coords'].append(image_info['coords'])
    IMAGES_INFO['x_min'].append(image_info['x_min'])
    IMAGES_INFO['x_max'].append(image_info['x_max'])
    IMAGES_INFO['y_min'].append(image_info['y_min'])
    IMAGES_INFO['y_max'].append(image_info['y_max'])
    IMAGES_INFO['pet_ratio'].append(image_info['pet_ratio'])
    
    # Not Every Image can be Correctly Classified
    labels = image_info['labels']
    if len(set(labels)) == 1: # unanimous label
        IMAGES_INFO['label'].append(labels[0])
    elif len(set(labels)) > 1: # Get label with highest confidence
        IMAGES_INFO['label'].append(labels[0])
    else: # unknown label, yolo could not find pet
        IMAGES_INFO['label'].append('unknown')

# Add Image Info to Train
for k, v in IMAGES_INFO.items():
    test[k] = v

### Preprocess data

In [ ]:
# to process label column
train = pd.get_dummies(train, prefix='label', columns=['label'])
test = pd.get_dummies(test, prefix='label', columns=['label'])

In [ ]:
# make the number of columns same for the train and test data (test data only has 'unknown' column)

def make_columns(df):
    for i in range(3):
        if 'label_cat' not in df.columns:
            df['label_cat']=0

        elif 'label_dog' not in df.columns:
            df['label_dog']=0

        elif 'label_unknown' not in df.columns:
            df['label_unknown']=0
        
    return df

train = make_columns(train)
test = make_columns(test)

In [ ]:
# apply MinMaxScaler
train_columns = train.drop(['Id', 'path', 'coords', 'Pawpularity', 'R', 'G', 'B'], axis=1).columns 
test_columns = test.drop(['Id', 'path', 'coords'], axis=1).columns

minMaxScaler = MinMaxScaler()

train_final = pd.DataFrame()
train_final[train_columns] = pd.DataFrame(minMaxScaler.fit_transform(train.drop(['Id', 'path', 'coords', 'Pawpularity', 'R', 'G', 'B'], axis=1)))

test_final = pd.DataFrame()
test_final[test_columns] = pd.DataFrame(minMaxScaler.transform(test.drop(['Id', 'path', 'coords'], axis=1)))

In [ ]:
train_final[['Id', 'path', 'Pawpularity']] = train[['Id', 'path', 'Pawpularity']]
# train_final[['Id', 'path']] = train[['Id', 'path']]
test_final[['Id', 'path']] = test[['Id', 'path']]

In [ ]:
# split dataset
train, val= train_test_split(train_final, test_size=0.2, random_state=0)

In [ ]:
train = train[['Pawpularity']].join(train.drop('Pawpularity', axis=1))
val = val[['Pawpularity']].join(val.drop('Pawpularity', axis=1))

In [ ]:
def process_data(path, meta, augment=False, label=True):
    img = tf.io.decode_jpeg(tf.io.read_file(path), channels=3)
    img = tf.cast(img, dtype=tf.float32)
#     img = tf.image.central_crop(img, 1.0)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = keras.applications.efficientnet.preprocess_input(img)
#     img = img / 255.
    img = tf.cast(img, dtype=tf.float64)
    
    if augment:
        img = tf.image.random_flip_left_right(img)
#         img = tf.image.random_brightness(img, 0.1)
        img = tf.image.random_saturation(img, 0.95, 1.05)
        img = tf.image.random_contrast(img, 0.95, 1.05)
        
    if label:
        return (img, meta[1:]), meta[0]
    return (img, meta), 0

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
IMG_SIZE = 299
BATCH_SIZE = 32

In [ ]:
# train_ds = tf.data.Dataset.from_tensor_slices((train['path'], train.drop(['path', 'Id'], axis=1).astype(float))).map(lambda x,y: process_data(x, y, True)).batch(BATCH_SIZE).prefetch(AUTOTUNE)
# val_ds = tf.data.Dataset.from_tensor_slices((val['path'], val.drop(['path', 'Id'], axis=1).astype(float))).map(process_data).batch(BATCH_SIZE).prefetch(AUTOTUNE)
test_ds = tf.data.Dataset.from_tensor_slices((test_final['path'], test_final.drop(['path','Id'], axis=1).astype(float))).map(lambda x,y: process_data(x, y, False, False)).batch(BATCH_SIZE).prefetch(AUTOTUNE)

### Modeling
- Use effnetB7
- Use metadata by concatenating
- Use skip connection

In [ ]:
eff_model = keras.models.load_model('../input/d/ekaterinadranitsyna/keras-applications-models/EfficientNetB7.h5')
eff_model.trainable = False

def get_model():
    img_input = tfl.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    meta_input = tfl.Input(shape=(21,))

    X = eff_model(img_input)
    X = tfl.BatchNormalization()(X)
# kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4)
    con = tfl.concatenate([X, meta_input])
#     con= tfl.BatchNormalization()(con)
    con = tfl.Dropout(0.4)(con)  

    skip = tfl.Dense(64,  
                     kernel_initializer=tf.keras.initializers.GlorotUniform(),
                    kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(con)
    
    X = tfl.Dense(256, activation='relu', 
                  kernel_initializer=tf.keras.initializers.GlorotUniform(),
                 kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(con)
    X = tfl.Dropout(0.4)(X)  
    
    X = tfl.Dense(256, activation='relu', 
                  kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(X)
    X = tfl.Dropout(0.4)(X)
    
    X = tfl.Dense(64,  
                  kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4))(X)


    X = tfl.Add()([X, skip])
    X= tfl.BatchNormalization()(X)
    X = tfl.Activation('relu')(X)
#     X = tfl.Dropout(0.4)(X)
    
#     X = tfl.Dense(16, activation='relu')(X)
#     X = tfl.Dropout(0.4)(X)


    

    out = tfl.Dense(1)(X)

    model = keras.Model(inputs=[img_input, meta_input], outputs=out)
    
    return model

In [ ]:
model = get_model()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# early_stop = keras.callbacks.EarlyStopping(
#         patience=3,
#         restore_best_weights=True)

# lr_schedule = keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=1e-3,
#     decay_steps=100,
#     decay_rate=0.96,
#     staircase=True)

In [ ]:
# model = get_model()

# model.compile(keras.optimizers.Adam(learning_rate=lr_schedule), 
#             loss='mse', 
#             metrics=[keras.metrics.RootMeanSquaredError()])

# history = model.fit(train_ds,
#                validation_data=val_ds,
#                epochs=20,
#                workers=-1,
#                callbacks=[early_stop])

In [ ]:
k = 5
fold = KFold(k,shuffle=True)

In [ ]:
models = []
histories = []

for i, (t_ids, v_ids) in enumerate(fold.split(train)):
    
    keras.backend.clear_session()

    print("\n\n===========================================================================================\n")
    train_ds = tf.data.Dataset.from_tensor_slices((train.iloc[t_ids]['path'], train.iloc[t_ids].drop(['path', 'Id'], axis=1).astype(float)))\
    .map(lambda x,y: process_data(x, y, True)).batch(BATCH_SIZE).prefetch(AUTOTUNE)
    
    val_ds = tf.data.Dataset.from_tensor_slices((train.iloc[v_ids]['path'], train.iloc[v_ids].drop(['path', 'Id'], axis=1).astype(float)))\
    .map(process_data).batch(BATCH_SIZE).prefetch(AUTOTUNE)
    
    model = get_model()
    
    early_stop = keras.callbacks.EarlyStopping(
        patience=3,
        restore_best_weights=True)

    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-3,
        decay_steps=1000,
        decay_rate=0.96,
        staircase=True)
    
    model.compile(keras.optimizers.Adam(learning_rate=lr_schedule), 
            loss='mse', 
            metrics=[keras.metrics.RootMeanSquaredError()])
    
    history = model.fit(train_ds,
                   validation_data=val_ds,
                   epochs=20,
                   callbacks=[early_stop])
    
    models.append(model)
    histories.append(history)

In [ ]:
# preds = model.predict(test_ds)
preds = models[0].predict(test_ds)/k

for i in range(1,k):
    preds += models[i].predict(test_ds)/k

In [ ]:
preds

In [ ]:
test['Pawpularity'] = preds
test[['Id', 'Pawpularity']].to_csv('submission.csv', index=False)